In [ ]:
# ===== IMPORTACION (EJECUTAR SIEMPRE) ===== #
import datetime
import os
import pandas as pd
import sys
import time


# ME TRAIGO MI CLASE
from AxesorMining import AxesorMining


print("### --- Versiones --- ###")
print("Python:", sys.version)
print("Pandas:", pd.__version__)
print("### ----------------- ###")

#### OBTENCION DE TODAS LAS EMPRESAS DE AXESOR _(Con dos cojones)_

Ideas generales:

- Recorro la carpeta de provincias.
- Dentro de cada archivo, recorro los municipios.
- Dentro de cada municipio, empieza la fiesta:<br>

  En el municipio aparece un listado paginado que parte de 1. En el directorio que generemos insertaremos el número de página para retomar el minado en caso de parada.

  En el caso de que solo exista una página, hay que puentear la iteración.

  El bucle debe completar todas las empresas de la página antes de pasar a la siguiente.
  Debemos leer el total de páginas para saber cuando parar.

  Una vez dentro de la página de la empresa, extraemos todos los datos. Cabe la posibilidad de que la empresa esté extinta, por lo que validamos para pasar de largo de ese registro.

- SEGURIDAD: Doy un sleep de 3 seg. después de cada REQUEST para evitar el CAPTCHA

---


##### DECLARO LAS RUTAS

In [ ]:
ROOT_PROVINCIAS = 'data/0_Provincias'
ROOT_DIRECTORIOS = 'data/1_Directorios'

# DEFINO DIRECTORIO POR DEFECTO PARA EL PRIMER BUCLE
lista_provincias = os.listdir(ROOT_PROVINCIAS)

### Y AHORA, SUJETAME EL CUBATA:

In [ ]:
for dir_provincia in lista_provincias:

    # ========== VARIABLES PARA LA PROVINCIA ========== #

    index_localidad = 0
    num_pagina = 1
    index_pagina = 0

    # ========== RUTAS ========== #

    # COMPONGO LAS RUTAS DE ESTA ITERACION
    path_file = ROOT_PROVINCIAS + '/' + dir_provincia
    path_directorio = ROOT_DIRECTORIOS + '/Empresas_' + dir_provincia

    print('>> Cargando ' + path_file)
    print(datetime.datetime.now())

    # ========== DATAFRAME DE LA PROVINCIA ========== #

    # LEO EL CSV DE LA PROVINCIA Y LO METO EN DATAFRAME
    df_provincia = pd.read_csv(path_file, sep=";")

    # EVALUO SI EL DIRECTORIO DE EMPRESAS YA EXISTE Y TIENE ALGUN REGISTRO
    if (os.path.exists(path_directorio) and len((pd.read_csv(path_directorio, sep=";")).index) != 0):

        # SI EXISTE TENGO QUE PESCAR EL ULTIMO VALOR

        # OBTENER:
        # - index_localidad
        # - num_pagina
        # - index_pagina

        # LEO EL CSV DEL DIRECTORIO DE EMPRESAS Y LO METO EN DATAFRAME
        df_directorio = pd.read_csv(path_directorio, sep=";")
        index_last = df_directorio.index[-1]

        # SOBRESCRIBO LAS VARIABLES EN EL PUNTO DONDE LO RETOMO
        index_localidad = int(df_directorio.loc[index_last]['Index Localidad'])
        num_pagina = int(df_directorio.loc[index_last]['Pagina'])

        # EL PRIMER REGISTRO QUE INSERTO
        index_pagina = int(df_directorio.loc[index_last]['Index Pagina'] + 1)

        # BORRO DE LA PROVINCIA ESOS MUNICIPIOS: YA ESTÁN SACADOS.
        df_provincia.drop(
            index=df_provincia.index[:index_localidad], axis=0, inplace=True)

    else:  # SI NO EXISTE, LO CREO COMO DE COSTUMBRE Y NO ALTERO LAS VARIABLES.

        # CREO EL CSV DE LA PROVINCIA VACIO PARA IR AÑADIENDO
        df = pd.DataFrame([], columns=AxesorMining().COLUMNAS_DF)
        df.to_csv(path_directorio, sep=';', index=False)

    # ===================================================== #
    # ================= EMPIEZA EL MINADO ================= #
    # ===================================================== #

    # RECORRO LOS MUNICIPIOS DEL DATAFRAME PROVINCIAL
    for index, row in df_provincia.iterrows():

        # ========== VARIABLES ========== #
        nombre_provincia = row['Provincia']
        nombre_municipio = row['Municipio']
        url_municipio = row['URL']
        url_base = url_municipio[:-1]  # OJO! -> BORRO LA PAGINACION

        print('>> Accediendo al directorio de ' + nombre_municipio)

        # ========== PAGINADO ========== #
        # NUEVO MÉTODO -> OBTENGO CUANTAS PÁGINAS DE RESULTADOS HAY EN ESE MUNICIPIO
        page_max = AxesorMining().paginado(url_municipio)

        # CREO UN BUCLE CON LAS URL SEGUN PAGINACION.
        # EL RANGO VA DE 'NUM_PAGINA' A 'PAGE_MAX' (+1 PARA QUE SEA INCLUIDA)

        # ========== RECORRO LAS PÁGINAS ========== #
        for page in range(num_pagina, int(page_max)+1):

            # EMPIEZO A CRONOMETRAR LA PÁGINA
            inicio_pagina = time.time()

            # COMPONGO LA URL
            print('\n>> -- Página ', page, ' --')
            url_page = url_base + str(page)
            print(url_page)

            # SACO EL LISTADO DE EMPRESAS DE LA PAGINA
            listado_empresas = AxesorMining().listado_empresas(url_page)

            # ME CARGO DEL LISTADO LAS EMPRESAS QUE YA EXISTEN
            del listado_empresas[0:index_pagina]

            # ========== RECORRO LAS EMPRESAS DE LA PÁGINA ========== #
            for url_empresa in listado_empresas:

                # DECLARO LA LISTA QUE HARÁ DE FILA EN LA TABLA
                data_empresas_municipio = []

                # EMPAQUETO Y LLAMO AL MÉTODO DE EXTRACCIÓN
                url_empresa = 'https:' + url_empresa.get('href').strip()
                parametros = [url_empresa, dir_provincia,
                              nombre_municipio, page, index_pagina, index_localidad]

                try:
                    insert_empresa = AxesorMining().obtener_empresa(parametros)
                    data_empresas_municipio.append(insert_empresa)
                    df = pd.DataFrame(data_empresas_municipio)
                    df.to_csv(path_directorio, mode='a',
                              sep=';', index=False, header=False)
                    print('>> Empresa añadida: ' + insert_empresa['Nombre'])
                except:
                    print("Empresa errónea")
                    continue                # AÑADO AL DATAFRAME E INSERTO EN CSV

                # PASO A LA SIGUIENTE POSICIÓN
                index_pagina += 1

            # PARO DE CRONOMETRAR LA PÁGINA
            fin_pagina = time.time()
            tiempo_pagina = fin_pagina - inicio_pagina

            print('----------------------- FIN DE LA PÁGINA --------------------------')
            print('---------------- EXTRAIDA EN ' +
                  str(tiempo_pagina) + ' -----------------')

            # RESETEO EL INDEX_PAGINA
            index_pagina = 0

        # SUMO AL INDEX DEL MUNICIPIO
        index_localidad += 1
        # RESETEO EL NUMERO DE PAGINA
        num_pagina = 1

        print('>> Municipio finalizado')
        print('>> ---')
        print(datetime.datetime.now(), '\n')

    print(datetime.datetime.now())

    print('>> --- PROVINCIA FINALIZADA --- ')
